In [1]:
import json
import io
import sys

from os.path import join

sys.path.append('../..')
from squad_tools import load_dict

### Read data

In [2]:
squad_neg_path = '/pio/data/data/squad/'

with open(join(squad_path, 'negative_samples')) as f:
    train = json.load(f)
    
with open(join(squad_path, 'dev-v1.1.json')) as f:
    dev = json.load(f)
    


In [ ]:
data_neg

## Preprocess wikipedia negative examples

In [171]:
for i in range(len(data_neg)):
    data_neg[i][1] = map(lower, word_tokenize(data_neg[i][1]))
    data_neg[i][2] = map(lower, word_tokenize(data_neg[i][2]))

In [172]:
for i in range(len(data_neg)):
    data_neg[i][2].append(not_a_word)

In [173]:
for i in range(len(data_neg)):
    data_neg[i][0] = [[len(data_neg[i][2]) - 1]]

In [174]:
train_wiki_neg_bin_feats = []

for _, q, c in data_neg:
    train_wiki_neg_bin_feats.append(make_bin_feats([q, c]))

In [179]:
train_wiki_neg_ascii_chars = []

chars = [unichr(i) for i in range(128)]
i_to_c = chars
c_to_i = {v:k for (k,v) in list(enumerate(chars))}

for _, q, x in data_neg:
    q_char = [[1] + [c_to_i.get(c, 0) for c in w] + [2] for w in q]
    x_char = [[1] + [c_to_i.get(c, 0) for c in w] + [2] for w in x[:-1]] + [[1, 3, 2]]
    train_wiki_neg_ascii_chars.append([q_char, x_char])

In [192]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_bin_feats, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_ascii_chars, f)

In [204]:
train_neg_with_glove_vocab = []

for a, q, x in data_neg:
    q_num = [glove_w_to_i.get(w, 0) for w in q]
    x_num = [glove_w_to_i.get(w, 0) for w in x]
    train_neg_with_glove_vocab.append([a, q_num, x_num])

In [206]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl', 'w') as f:
    pickle.dump(train_neg_with_glove_vocab, f)

In [2]:
train_wiki_neg_bin_feats = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl')
train_wiki_neg_ascii_chars = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl')
train_neg_with_glove_vocab = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl')

In [6]:
too_long = []

for i, (_, x) in enumerate(train_wiki_neg_ascii_chars):
    for w in x:
        if len(w) > 35:
            too_long.append(i)
            break

In [8]:
train_wiki_neg_bin_feats_cut = []
train_wiki_neg_ascii_chars_cut = []
train_neg_with_glove_vocab_cut = []

for i in xrange(len(train_neg_with_glove_vocab)):
    if i not in too_long:
        train_wiki_neg_bin_feats_cut.append(train_wiki_neg_bin_feats[i])
        train_wiki_neg_ascii_chars_cut.append(train_wiki_neg_ascii_chars[i])
        train_neg_with_glove_vocab_cut.append(train_neg_with_glove_vocab[i])

In [10]:
len(train_neg_with_glove_vocab_cut)

84833

In [11]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_bin_feats_cut, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_ascii_chars_cut, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl', 'w') as f:
    pickle.dump(train_neg_with_glove_vocab_cut, f)